In [2]:
from decimal import Decimal, getcontext
import numpy as np
import plotly.express as px
import pandas as pd
from scipy import integrate

# Установим точность вычислений Decimal
getcontext().prec = 100

# Константы
c = Decimal(3) * Decimal(1e10)    # Скорость света
R = Decimal('0.35')               # Радиус цилиндра

# температурное поле в цилиндре
def T_z(z):
    Tw, To, p = Decimal(2000), Decimal(1e4), Decimal(4)
    return (Tw - To) * (z ** p) + To

# функция Планка
def up_z(z):
    numerator = Decimal('3.084') * Decimal(1e-4)
    denominator = (Decimal('4.799') * Decimal(1e4) / T_z(z)).exp() - Decimal(1)
    return numerator / denominator

def read_data_kT(variant=1):
    data_tab = []
    with open('data/kT.txt') as file:
        line = file.readline()
        while line != '':
            line = line.split()
            if variant == 1:
                data_tab.append([int(line[1]), Decimal(line[2])])
            else:
                data_tab.append([int(line[1]), Decimal(line[3])])
            line = file.readline()
    return data_tab

def getk_zFunc(variantkT):
    data_kT = read_data_kT(variantkT)
    Tvals = [Decimal(data_kT[i][0]).ln() for i in range(len(data_kT))]
    kvals = [Decimal(data_kT[i][1]).ln() for i in range(len(data_kT))]
    
    def k_z(z):
        T = T_z(z)
        log_T = T.ln()
        log_k = np.interp([float(log_T)], [float(t) for t in Tvals], [float(k) for k in kvals])[0]
        return Decimal(log_k).exp()
    
    return k_z

def getStartParams(h, k_z, z0):
    denominator = R**2 * (k_z(z0 + h) + k_z(z0))
    M0 = Decimal(1) / denominator
    K0 = -M0
    Q0 = Decimal(0)
    epsilon1 = -K0 / M0
    eta1 = Q0 / M0
    print(f'M0 = {M0}, K0 = {K0}')
    print(f'epsilon1 = {epsilon1}, eta1 = {eta1}')
    return epsilon1, eta1

def vn(zn, h):
    return zn * h

def kappa_n_p_half(zn, znp1, k_z):
    return Decimal(2) / (k_z(znp1) + k_z(zn))

def kappa_n_m_half(zn, znm1, k_z):
    return Decimal(2) / (k_z(znm1) + k_z(zn))

def An(zn, h, k_z):
    numerator = (zn - h / Decimal(2)) * kappa_n_m_half(zn, zn - h, k_z)
    return numerator / (R**2 * h)

def Dn(zn, h, k_z):
    numerator = (zn + h / Decimal(2)) * kappa_n_p_half(zn, zn + h, k_z)
    return numerator / (R**2 * h)

def Bn(zn, h, k_z):
    return An(zn, h, k_z) + Dn(zn, h, k_z) - Decimal(3) * k_z(zn) * vn(zn, h)

def Fn(zn, h, k_z):
    return -Decimal(3) * k_z(zn) * up_z(zn) * vn(zn, h)

def getyN(epsilonN, etaN, h, k_z):
    kappa = Decimal(2) / (k_z(Decimal(1)-h) + k_z(Decimal(1)))
    MN = -(Decimal(1) - h / Decimal(2)) * kappa / (R**2 * h)
    KN = -MN + Decimal(3) * Decimal('0.39') / R
    QN = Decimal(0)
    yN = (QN - KN * etaN) / (KN * epsilonN + MN)
    print(f'epsilonN={epsilonN}, etaN={etaN}')
    print(f'kappa={kappa}, KN={KN}, MN={MN}')
    return yN

def epsilon_eta_ip1(zi, epsiloni, etai, h, k_z, i):
    Ai = An(zi, h, k_z)
    Bi = Bn(zi, h, k_z)
    Di = Dn(zi, h, k_z)
    Fi = Fn(zi, h, k_z)
    denominator = Bi - Ai * epsiloni
    epsilon_ip1 = Di / denominator
    eta_ip1 = (Ai * etai + Fi) / denominator
    print(f'i={i}, zi={zi}: ', end='')
    print(f'epsilon={epsilon_ip1}, eta={eta_ip1}')
    return (epsilon_ip1, eta_ip1)

def progonka(k_z, N=100):
    N -= 1
    z0, zN = Decimal(0), Decimal(1)
    h = (zN - z0) / Decimal(N)
    print(f'h = {h}')
    epsilon1, eta1 = getStartParams(h, k_z, z0)
    zvals = [z0]
    epsilonVals, etaVals = [epsilon1], [eta1]

    for i in range(N):
        epsilonNext, etaNext = epsilon_eta_ip1(zvals[-1], epsilonVals[-1], etaVals[-1], h, k_z, i)
        epsilonVals.append(epsilonNext)
        etaVals.append(etaNext)
        zvals.append(zvals[-1] + h)

    yN = getyN(epsilonVals[-1], etaVals[-1], h, k_z)
    print(f'yN = {yN}')
    yvals = [Decimal(0)] * (N + 1)
    yvals[N] = yN
    for i in range(N - 1, -1, -1):
        yi = epsilonVals[i + 1] * yvals[i + 1] + etaVals[i + 1]
        yvals[i] = yi
    
    return yvals, zvals

def getFvals(zvals, yvals, h, k_z):
    upvals = [up_z(z) for z in zvals]
    Fvals = [Decimal(0)]
    intFunc = [Decimal(0)]
    
    for i in range(1, len(zvals)):
        integrand = zvals[i] * k_z(zvals[i]) * (upvals[i] - yvals[i])
        intFunc.append(integrand)
        # Используем составное правило трапеций для интегрирования
        integral = Decimal(0)
        for j in range(1, i+1):
            integral += (intFunc[j-1] + intFunc[j]) * (zvals[j] - zvals[j-1]) / Decimal(2)
        Fvals.append(Decimal(3) * R / zvals[i] * integral)
    
    return Fvals

def get_dFvals(zvals, uvals, upvals, Fvals, k_z):
    dF = []
    for i in range(len(zvals)):
        term1 = Decimal(3) * R * k_z(zvals[i]) * (upvals[i] - uvals[i])
        term2 = Fvals[i] / zvals[i] if zvals[i] != Decimal(0) else Decimal(0)
        dF.append(term1 - term2)
    return dF

# Основная часть программы
cntSteps = 1000
k_z = getk_zFunc(variantkT=2)
uvals, zvals = progonka(k_z, cntSteps)
upvals = [up_z(z) for z in zvals]
Fvals = getFvals(zvals, uvals, zvals[1]-zvals[0], k_z)
dFvals = get_dFvals(zvals, uvals, upvals, Fvals, k_z)

# Преобразуем Decimal в float для построения графиков
zvals_float = [float(z) for z in zvals]
uvals_float = [float(u) for u in uvals]
upvals_float = [float(up) for up in upvals]
Fvals_float = [float(f) for f in Fvals]
dFvals_float = [float(df) for df in dFvals]

# Создаем DataFrame для графиков
df1 = pd.DataFrame({"x": zvals_float, "y": uvals_float, "label": ["u(z)"] * len(uvals_float)})
df2 = pd.DataFrame({"x": zvals_float, "y": upvals_float, "label": ["u_p(z)"] * len(upvals_float)})
df3 = pd.DataFrame({"x": zvals_float, "y": Fvals_float, "label": ["F(z)"] * len(Fvals_float)})
df4 = pd.DataFrame({"x": zvals_float, "y": dFvals_float, "label": ["dF(z)"] * len(dFvals_float)})

# Объединяем DataFrame для первого графика
df_combined = pd.concat([df1, df2])
fig1 = px.line(df_combined, x="x", y="y", color="label", title="", markers=True)
fig1.show()

# График для F(z)
fig2 = px.line(df3, x="x", y="y", color="label", title="", markers=True)
fig2.show()

# График для dF(z)
fig3 = px.line(df4, x="x", y="y", color="label", title="", markers=True)
fig3.show()

h = 0.001001001001001001001001001001001001001001001001001001001001001001001001001001001001001001001001001001
M0 = 0.02040816326533076327893292184194919085349392341371384058422576378918277486420986344383006341264113550, K0 = -0.02040816326533076327893292184194919085349392341371384058422576378918277486420986344383006341264113550
epsilon1 = 1, eta1 = 0E+101
i=0, zi=0: epsilon=1, eta=-0E-103
i=1, zi=0.001001001001001001001001001001001001001001001001001001001001001001001001001001001001001001001001001001: epsilon=1.009917010810324374205750847006655799212947616844912527294577499287945734433263012158098211939152753, eta=-2.540437208543842229655915469618290099470511571735024244569927451140392848871439658762941329014396838E-8
i=2, zi=0.002002002002002002002002002002002002002002002002002002002002002002002002002002002002002002002002002002: epsilon=1.018053925831332825135399796974624180024270193001448625868832313145344398011884289777858718668799940, eta=-4.624867897964086621456614845560315129944951